# HW2 Text Addition
Austin Gill

Goal: Use the GPS sensor feature of Veranda to implement live path tracing.

---

In the file `wiggler.py`, save the following:

```python
import rclpy as ros
from std_msgs.msg import Float32
from veranda.SimTimer import SimTimer


class Wiggler:
    """The wiggle code from the textbook, wrapped in a class to avoid
    contracting cancer from global variables.
    """

    def __init__(self):
        """Creates an instance of the Wiggler class."""
        self.node = ros.create_node('Wiggler')

        # Create a simulation timer so we can speed up the simulation without
        # breaking things.
        self.simulation = SimTimer(True, 'veranda/timestamp', self.node)
        self.timer = self.simulation.create_timer(1, self.wiggle_left)

        # Create publishers for the left and right wheels.
        self.left_pub = self.node.create_publisher(Float32, 'robot/left_wheel')
        self.right_pub = self.node.create_publisher(Float32, 'robot/right_wheel')

    def wiggle_left(self):
        """Wiggles the robot to the left"""
        self.simulation.destroy_timer(self.timer)
        msg = Float32()
        msg.data = 5.0
        self.right_pub.publish(msg)
        msg.data = 0.0
        self.left_pub.publish(msg)
        self.timer = self.simulation.create_timer(1, self.wiggle_right)

    def wiggle_right(self):
        """Wiggles the robot to the right"""
        self.simulation.destroy_timer(self.timer)
        msg = Float32()
        msg.data = 0.0
        self.right_pub.publish(msg)
        msg.data = 5.0
        self.left_pub.publish(msg)
        self.timer = self.simulation.create_timer(1, self.wiggle_left)
```

This is the wiggle code given previously as an example, but wrapped in a Python class.

Now in the file `tracer.py` save the following:

```python
from collections import deque

import matplotlib.pyplot as plt
import rclpy as ros
from geometry_msgs.msg import Pose2D


class PathTracer:
    """Subscribes to Pose2D messages on /robot/gps and plots the simulated path
    traced by a robot in Veranda.
    """

    def __init__(self, max_history=1000):
        """Creates an instance of the PathTracer class.

        :param max_history: The maximum number of points to plot. The oldest
            points will be removed from the plot once the PathTracer accumulates
            this many points.
        """
        self.node = ros.create_node('PathTracer')
        self.subscriber = self.node.create_subscription(Pose2D, '/robot/gps', self.callback)
        # Create a matplotlib figure
        plt.figure()
        # Create an empty plot. Note the comma!! plt.plot() returns a list with one item in it.
        self.plot, = plt.plot([])
        plt.title('Simulated Robot Path')
        # Turn on interactive plotting
        plt.ion()
        # Display the plot
        plt.show()

        # Memory isn't free, so save only the latest max_history points in a deque.
        # Use two deques, one for the x points, one for the y points to simplify plotting.
        self.xs = deque([], maxlen=max_history)
        self.ys = deque([], maxlen=max_history)

    def callback(self, msg):
        """Receives Pose2D messages from the robot and plots them."""
        # Save the received point in a tuple, and enqueue it in self.points.
        self.xs.append(msg.x)
        self.ys.append(msg.y)

        # Now update the plot now that we have a new point.
        self.update_plot()

    def update_plot(self):
        """Updates the live plot with new points."""
        # Set the plot's data to the new x and y points.
        self.plot.set_data(self.xs, self.ys)

        # Now expand the plot's axis boundary as the path grows.
        self.plot.axes.set_xlim(min(self.xs) - 0.1, max(self.xs) + 0.1)
        self.plot.axes.set_ylim(min(self.ys) - 0.1, max(self.ys) + 0.1)

        # This is the magical bit that causes matplotlib to update the plot live.
        plt.pause(0.000001)
```

This creates a class to receive data from the robot GPS on the `/robot/gps` topic and produce a live `matplotlib` plot.

Now in the file `main.py` save the following:

```python
#!/usr/bin/env python3
import rclpy as ros
from rclpy.executors import SingleThreadedExecutor

from tracer import PathTracer
from wiggler import Wiggler


def main():
    """A single main entry point for our ROS nodes."""
    # Initialize the ROS client.
    ros.init()

    # Create instances of our Wiggle and PathTracer classes.
    wiggler = Wiggler()
    tracer = PathTracer()

    # A SingleThreadedExecutor executes both nodes in the same thread. There
    # is a MultiThreadedExecutor, but it doesn't play nicely with matplotlib.
    executor = SingleThreadedExecutor()

    # Add the ROS nodes to the executor.
    executor.add_node(wiggler.node)
    executor.add_node(tracer.node)

    try:
        # Run the executor, which in turn runs the nodes.
        executor.spin()
    except KeyboardInterrupt:
        # Destroy the nodes and shutdown the ROS client when the user hits <ctrl-c>
        tracer.node.destroy_node()
        wiggler.node.destroy_node()
        ros.shutdown()


if __name__ == '__main__':
    main()
```

This is a single main entry point for both of the ROS nodes we just created. The use of the `SingleThreadedExecutor` will execute both ROS nodes in the same thread, but switch back and forth between them. There is a `MultiThreadedExecutor` that will run each node in its own thread, but matplotlib doesn't play nicely with threads.

---

Now in Veranda, create a differential drive robot with a GPS sensor centered on the body of the robot. The author used the following configuration:

* The GPS sensor publishes messages at 10Hz on the topic `/robot/gps`
* The left and right wheels are driven, have a density of 10, and listen for their speeds on the topics `/robot/left_wheel` and `/robot/right_wheel` respectively.

Add your robot to a new simulation, and press "Play". As usual, the robot will do nothing as we have not given the left or right wheels a speed yet.

From a new terminal (with the ROS environment `source`d), run `ros2 topic list`. You should see the following topics listed.
```
/clock
/parameter_events
/robot/gps
/robot/left_wheel
/robot/right_wheel
/veranda/timestamp
```
Now run `ros2 topic echo /robot/gps`. You should see a *bunch* of messages that look like the following
```
x: 4.66600635589806e-310
y: 4.66600635589806e-310
theta: 4.66600635589806e-310

x: 4.66600635589806e-310
y: 4.66600635589806e-310
theta: 4.66600635589806e-310

...
```
This is the data from the GPS sensor on the robot. Now run the `main.py` file with the command `python3 main.py`. A live matplotlib window should pop up showing the path of the robot as it moves through the workspace. It should look like the following:

![Wiggle Path](addition/path.svg)

Now click the "Save" button in the matplotlib window, and watch things break. After saving the image, you might see something like the following

![thread blocking path](addition/path2.svg)

What happened!?

This is a consequence of running the matplotlib GUI *and* the `Wiggler` node *in the same thread*. So when you pressed "Save", you blocked the execution of the entire thread, blocking the regular wheel speed updates every second. This is a limitation of using the `rclpy.executors.SingleThreadedExecutor` to execute the ROS nodes. As mentioned above, there *is* a `MultiThreadedExecutor`, but matplotlib requires its GUI event loop to be ran in the main thread in order to show the plot window. At this time, the author is not aware of a solution.